In [39]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown

import warnings
warnings.filterwarnings('ignore')

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Расчет сведений для МОНИТОРИНГА по СДВ, НС и ТРАНСМУК, по сведениям ЦЛО  <br>
</b>            </div>

### 
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Функции для расчета  <br>
</b>            </div>

#### функция 605го

In [40]:
def load_data_605(file_name, bmchu, bk, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------
    
    # file_name = file_605
    df = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    df_605 = df.copy()
    df_605 = df_605[['Дата постановки на учет/включения пациента в реестр',
           'Пациент', 
           'Дата рождения', 'Пол',
           'Дата признания пациента нуждающимся в ПМП (ВК)',
           'Текущий статус пациента',
           'Дата прекращения оказания ПМП', 
             'Курирующее подразделение/филиал',
                   'Гражданский статус' ,
                    'Адрес', 'Основной диагноз по МКБ-10',
                     'Адрес фактический',
                     'Дата разрешения ДЗМ',
                     'Номер разрешения ДЗМ',
                     'БК'
                    ]]

    #  переименуем покороче столбцы
    df_605.rename(columns={'Дата постановки на учет/включения пациента в реестр': 'Дата_вкл_реестр'}, inplace=True)
    df_605.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
    df_605.rename(columns={'Дата признания пациента нуждающимся в ПМП (ВК)': 'Дата_признания'}, inplace=True)
    df_605.rename(columns={'Дата прекращения оказания ПМП': 'Дата_прекр'}, inplace=True)
    df_605.rename(columns={'Курирующее подразделение/филиал': 'Филиал'}, inplace=True)
    df_605.rename(columns={'Основной диагноз по МКБ-10': 'Диагноз'}, inplace=True)
    df_605.rename(columns={'Гражданский статус': 'Иногород'}, inplace=True)

    df_concat = pd.concat([df_605])


    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()


    # Уберем записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'
    # - удаляем строки. в которых текущий статус не заполнен =  nan
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isin(['Новый пациент', 'Умер без п/о', 'Вне реестра', 'Врач КЦ']))]   # -----------------записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'----------------
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isna())]  # - удаляем строки. в которых текущий статус не заполнен =  nan


    # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bmchu != 'учитываем':
        df_concat = df_concat.loc[df_concat['Филиал'] != 'БМЧУ "Детский хоспис"']  # - удаляем строки. в которых текущий статус не заполнен =  nan
        
        
   #     # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bk != 'с учетом БК':
        df_concat = df_concat.loc[df_concat['БК'] != 'Да']  # - удаляем строки. в которых текущий статус не заполнен =  nan     

    # УДАЛИМ полные дубликаиы строк (оставляем только ПЕРВЫЕ строки из повторов (из 605го) )
    df_concat = df_concat.drop_duplicates(keep='first')
    df_concat.index.duplicated()
    df_concat.reset_index(drop= True , inplace= True )


    # Заполним поле Дата_первых_сведений данными: 
    #  если Дата_вкл_реестр не пустая, то  Дата_учета = Дата_вкл_реестр

    df_concat.loc[ (~df_concat['Дата_вкл_реестр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_вкл_реестр']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (~df_concat['Дата_признания'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_признания']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (df_concat['Дата_признания'].isnull()) & (~df_concat['Дата_прекр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_прекр'] - pd.Timedelta(days = 2)



    # новый признак - Дата_выбытия

    # 1 -- условие ---------------------------
    df_concat.loc[ (  
               ((df_concat['Дата_прекр'].isnull()) & (df_concat['Дата_вкл_реестр'] < '2020-12-31'))  
            &  (  (df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП') )
               )
               , 'Дата_выбытия'] = '01.01.1900'

    # 2 -- условие----------------------------
    df_concat.loc[(~df_concat['Дата_прекр'].isnull()), 'Дата_выбытия'] = df_concat['Дата_прекр']

    # 3 -- условие ---------------------------
    df_concat.loc[
                     (~df_concat['Дата_первых_сведений'].isnull() & df_concat['Дата_выбытия'].isnull() )  
                  &  ((df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП')), 'Дата_выбытия'] = df_concat['Дата_первых_сведений'] + pd.Timedelta(days = 2)



    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_выбытия'] = pd.to_datetime(df_concat['Дата_выбытия'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_первых_сведений'] = pd.to_datetime(df_concat['Дата_первых_сведений'], dayfirst=True, errors = 'coerce').dt.normalize()


    # ------------подсчитаем ВОЗРАСТ --------------

    
   # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
    df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_concat['ДР']).astype('<m8[Y]')




    #  ------ посчитаем категорию трудоспособный --------
    #  если пол ЖЕНСКИЙ, и возраст Менее 57 -----> Категория - трудоспособный
    #  если пол Мужской, и возраст Менее 62 -----> Категория - трудоспособный

    df_concat['Категория'] = np.nan
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Мужской') & (df_concat['Возраст'] >= 62), 'Категория'] = 'Старше трудосп'
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Женский') & (df_concat['Возраст'] >= 57), 'Категория'] = 'Старше трудосп'


    #  ------ определим детей ---------------
    df_concat['Дети'] = np.nan
    df_concat.loc[(df_concat['Возраст'] < 18) & (df_concat['Возраст'] >= 0), 'Дети'] = 'Дети'



    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_concat['Ключ'] = df_concat['Пациент'].map(str) + (df_concat['ДР'].dt.date).map(str)
    df_concat['Ключ'] = df_concat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_concat['Ключ'] = df_concat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 


    df_clr = df_concat.loc[ (~df_concat['Дата_первых_сведений'].isnull())]

    # ------ удаление дубликатов ----------
    df_clr = df_clr.drop_duplicates(['Ключ'], keep='first') 
    
    return df_clr  

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования датафрейма ПРЕПАРАТА (универс)  <br></b>            </div>

In [41]:
def load_preparat(file_name, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла preparat ----------------------
    df_preparat = pd.read_csv(file_name, delimiter=';', encoding='cp1251')


        # ----------------------- Уберем строки, в которых нет  даных о выдаче препаратов --------  (препараты должны быть с 5й строки)------    
    df_preparat = df_preparat[df_preparat[df_preparat.columns[5:]].notna().any(axis=1)]
    
    
    # уберем лишние колонки
    df_preparat = df_preparat[['№', 'Серия и номер полиса ОМС', 'СНИЛС', 'ФИО пациента',   'Дата рождения']]
    
    

    # ---- преобразуем дату в тип дата -----------
    df_preparat['Дата рождения'] = pd.to_datetime(df_preparat['Дата рождения'], dayfirst=True, errors = 'coerce').dt.normalize()

    # ------ Добавим поле ВОЗРАСТ и ДЕТИ------------
    df_preparat['Возраст'] = np.nan
    df_preparat['Дети'] = np.nan
    # ------------подсчитаем ВОЗРАСТ --------------
  # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_preparat['Возраст'] = (now - df_preparat['Дата рождения']).astype('<m8[Y]')
    df_preparat.loc[df_preparat['Возраст'] < 18,  'Дети'] = 'Дети'

    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
    df_preparat['Ключ'] = df_preparat['ФИО пациента'].map(str) + (df_preparat['Дата рождения'].dt.date).map(str)
    df_preparat['Ключ'] = df_preparat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_preparat['Ключ'] = df_preparat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 

    # ----- Заполним новый признак о всех пацментах датафрейма сведениями о полученных препаратах (т.к. все они из файла ЦЛО о получивших трансмук)
    df_preparat['preparat'] = 'preparat'
    
    return df_preparat

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Количество пациентов, получателей помощи (чел.)  <br></b>            </div>

In [42]:
# 4444 ---------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients_data_vkl_reestr(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------

    df_all = df.loc[ ((df['Дата_первых_сведений'] < finish)  
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))]

    data_all_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)  
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))].shape[0] 


    data_all_clients_trud = df.loc[ ((df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)))  
                             &   (  df['Категория'] == 'Старше трудосп')        
                                  ].shape[0]
    data_all_clients_deti = df.loc[ ((df['Дата_первых_сведений'] < finish) 
                     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)))  
                     # &   (  df['Категория'] == 'Старше трудосп')          
                    &   (  ~df['Дети'].isna())          
                                  
                                  ].shape[0]
    
    
    return  df_all, data_all_clients, data_all_clients_trud, data_all_clients_deti, data_all_clients 

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция Получатели (универсальная) <br></b>            </div>

In [43]:
def poluchateli_v_reestre(name_605, name_preparat, start, finish, bmchu, bk):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, bk, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()

    

    return df_605_period.shape[0], df_605_period_trud, df_preparat_vzros.shape[0], df_preparat_vzros_trud.shape[0], df_preparat_deti.shape[0], def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[3], df_preparat_deti
    

<hr>

<font color= green size=10>__ВЫВОД РЕЗУЛЬТАТОВ__
<hr>


### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Статистика СДВ НС и трансмук <br></b>            </div>

In [142]:
# ---------------------------- СДВ ---------------
start = '2023-01-01'
finish = '2023-07-01'


# bmchu = 'учитываем'
bmchu = 'не учитываем'
# bk = 'с учетом БК'
bk = 'без учета БК'

# -----1й квартал ----------
# file_name = '605_рц_бк_1204.csv'
# file_preparat_sdv = './ЦЛО/Форма_ВЛП-21_Выбранные_МНН_Сильнродействующие_01_03_2023_work_&_НСиПВ.csv'
# file_preparat_ns = './ЦЛО/Форма_ВЛП-21_Выбранные_МНН_Наркотические ЛП _01_03_2023_work_&_НСиПВ.csv'
# file_preparat_transmuk = './ЦЛО/Форма_ВЛП-21_Выбранные_Психотропные_3МНН_01_03_2023_work.csv'

# -----2й квартал ----------
file_name = '605_рц_бк_0407_.csv'
file_preparat_sdv = './ЦЛО/Сильнодействующие ЛП_30_06_2023_подгот.csv'
file_preparat_ns = './ЦЛО/Наркотические_ЛП_30_06_2023_подгот.csv'
file_preparat_transmuk = './ЦЛО/Психотропные_ЛП_3 МНН_30_06_2023_подгот.csv'



god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]

new_data_sdv = poluchateli_v_reestre(file_name, file_preparat_sdv, start, finish, bmchu, bk)
new_data_ns = poluchateli_v_reestre(file_name, file_preparat_ns, start, finish, bmchu, bk)
new_data_trans = poluchateli_v_reestre(file_name, file_preparat_transmuk, start, finish, bmchu, bk)

a = f'Статистика СДВ НС Трансмуказальные в период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f''' # {a}

| Параметр | Количество| В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|
| Количество пациентов в реестре за период __{bk}__      | {new_data_sdv[0]}   | {new_data_sdv[1]}   | {new_data_sdv[5]} |
| Количество пациентов, получивших __СДВ__               | {new_data_sdv[2]}   | {new_data_sdv[3]}   |{new_data_sdv[4]}|
| Количество пациентов, получивших __НС__                | {new_data_ns[2]}    | {new_data_ns[3]}    |{new_data_ns[4]}|
| Количество пациентов, получивших __Трансмуказальные__  | {new_data_trans[2]} | {new_data_trans[3]} |{new_data_trans[4]}|

''', raw=True)

 # Статистика СДВ НС Трансмуказальные в период с 01.01.2023 по 01.07.2023:

| Параметр | Количество| В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|
| Количество пациентов в реестре за период __без учета БК__      | 20186   | 16761   | 513 |
| Количество пациентов, получивших __СДВ__               | 4101   | 3396   |1|
| Количество пациентов, получивших __НС__                | 2516    | 1933    |3|
| Количество пациентов, получивших __Трансмуказальные__  | 12 | 0 |68|



### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Количество пациентов, получателей помощи (чел.)  <br></b>            </div>

In [44]:
# 4444 ---------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients(df, start, finish):
    
 # ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------

    df_all = df.loc[ ((df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                    ]

    data_all_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)   &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                             ]

    data_all_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_all_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Категория'] == 'Старше трудосп')    

                                        ]
    
    data_all_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish) 
                         &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                         &   (  ~df['Дети'].isna()) 

                                  ]
    
    return data_all_clients.shape[0],  data_all_clients_vzros.shape[0], data_all_clients_vzros_trud.shape[0], data_all_clients_deti.shape[0], data_all_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета НОВЫХ пациентов, обратившихся в ЭТОМ периоде (чел.)  <br></b>            </div>

In [45]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)
def def_data_new_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------Дата_первых_сведений


    data_new_clients = df.loc[ ((df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) )

                             ]


    data_new_clients_vzros = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_new_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Категория'] == 'Старше трудосп') 

                                        ]
    
    data_new_clients_deti = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) 
                         &   (  ~df['Дети'].isna())    

                                  ]
    
    return data_new_clients.shape[0],  data_new_clients_vzros.shape[0], data_new_clients_vzros_trud.shape[0], data_new_clients_deti.shape[0], data_new_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета УМЕРШИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [83]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)--------------------------------------Дата_первых_сведений
def def_data_dead_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------


    data_dead_clients = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                              & (df['Текущий статус пациента'] == 'Умер')
                              ]


    data_dead_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   &   (  df['Дети'].isna())
                                    & (df['Текущий статус пациента'] == 'Умер')
                                   ]
    
    data_dead_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  ) 
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (df['Текущий статус пациента'] == 'Умер')
                                        ]
    
    data_dead_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   & (  ~df['Дети'].isna())                                     
                                   & (df['Текущий статус пациента'] == 'Умер')
                                  ]
    
    return data_dead_clients.shape[0],  data_dead_clients_vzros.shape[0], data_dead_clients_vzros_trud.shape[0], data_dead_clients_deti.shape[0], data_dead_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета ИНОГОРОДНИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [47]:
# 100723 ----------------------------------------------ИНОГОРОДНИЕ-----------------------
def def_data_inogorod_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'


    data_inogorod_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))
                             
                                   & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   ) ]


    
    data_inogorod_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )  
                                   &   (  df['Дети'].isna())
                                   & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )   
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ) 
                                   & (  ~df['Дети'].isna())                                     
                                  & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    return data_inogorod_clients.shape[0],  data_inogorod_clients_vzros.shape[0], data_inogorod_clients_vzros_trud.shape[0], data_inogorod_clients_deti.shape[0], data_inogorod_clients

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=6>
<b> РЕЗУЛЬТАТ Статистика МОНИТОРИНГ <br></b>            </div>

In [115]:
# ---------------------------- ИТОГИ ---------------
start = '2023-01-01'
finish = '2023-04-01'
# file_605 = '605_0404.csv'
# file_605 = '605_рц_бк_1204.csv'
file_605 = '605_рц_бк_0407_.csv'

bmchu = 'не учитываем'
# bmchu = 'учитываем'
# bk = 'с учетом БК'
bk = 'без учета БК'


god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]


# ------------------------------------------------готовим реестр ------
df_reest = load_data_605(file_605, bmchu, bk,finish)

# ------------------------------------------------собираем данные ------
all_clients  = def_data_all_clients(df_reest, start, finish)
new_clients  = def_data_new_clients(df_reest, start, finish)
dead_clients = def_data_dead_clients(df_reest, start, finish)
inogorod_clients = def_data_inogorod_clients(df_reest, start, finish)
# data_inogorod_clients_all = def_data_inogorod_clients_all(df_reest, start, finish)

a = f'Статистика МОНИТОРИНГ период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f''' # {a}

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__{bmchu}__ и __{bk}__| {all_clients[0]}      | {all_clients[1]} |{all_clients[2]} |{all_clients[3]} |
| Количество __НОВЫХ__ пациентов за период          | {new_clients[0]}      | {new_clients[1]} |{new_clients[2]} |{new_clients[3]} |
| Количество __УМЕРШИХ__ пациентов за период        | {dead_clients[0]}     | {dead_clients[1]} |{dead_clients[2]} |{dead_clients[3]} |
| Количество __ИНОГОРОДНИХ__ пациентов за период    | {inogorod_clients[0]} | {inogorod_clients[1]} |{inogorod_clients[2]} |{inogorod_clients[3]} |
| в __ИНОГОРОДНИХ__ __{bk}__ учтены:                | ---- |---- |---- |---- |
| __'Гражданин РФ/иногородний', 'ЛНР', 'ДНР'__      | ---- |---- |---- |---- |


''', raw=True)

 # Статистика МОНИТОРИНГ период с 01.01.2023 по 01.04.2023:

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__не учитываем__ и __без учета БК__| 15582      | 15100 |12796 |482 |
| Количество __НОВЫХ__ пациентов за период          | 4393      | 4342 |3731 |51 |
| Количество __УМЕРШИХ__ пациентов за период        | 3653     | 3637 |3159 |16 |
| Количество __ИНОГОРОДНИХ__ пациентов за период    | 73 | 66 |39 |7 |
| в __ИНОГОРОДНИХ__ __без учета БК__ учтены:                | ---- |---- |---- |---- |
| __'Гражданин РФ/иногородний', 'ЛНР', 'ДНР'__      | ---- |---- |---- |---- |




In [116]:
# df_dead_1204 = dead_clients[4]
# df_dead_1204['period'] = 'apr'
# df_dead_1204
df_dead_0407 = dead_clients[4]
df_dead_0407['period'] = 'jul'
df_dead_0407

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ,period
0,2023-02-02,Зинатшина Анастасия Владимировна,1986-09-22,Женский,2023-01-31,Умер,2023-02-03,Зеленоград ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",...,NaN,NaN,Нет,2023-02-02,2023-02-03,36.0,NaN,NaN,зинатшинаанастасиявладимировна1986-09-22,jul
4,2023-03-03,Комарова Анастасия Анатольевна,1978-01-17,Женский,2023-03-01,Умер,2023-03-05,Зеленоград ОВПП,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",...,NaN,NaN,Нет,2023-03-03,2023-03-05,45.0,NaN,NaN,комароваанастасияанатольевна1978-01-17,jul
16,2023-03-19,Ямщикова Нина Ивановна,1969-03-22,Женский,2023-03-07,Умер,2023-03-27,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 847, квартира 83",...,NaN,NaN,Нет,2023-03-19,2023-03-27,54.0,NaN,NaN,ямщикованинаивановна1969-03-22,jul
19,2023-03-27,Кочетов Сергей Владимирович,1968-11-30,Мужской,2023-03-22,Умер,2023-03-28,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 1505, квартира 72",...,NaN,NaN,Нет,2023-03-27,2023-03-28,54.0,NaN,NaN,кочетовсергейвладимирович1968-11-30,jul
43,2023-02-12,Макаркина Галина Владимировна,1961-05-06,Женский,2023-02-03,Умер,2023-03-04,Зеленоград ОВПП,Москвич,"124681, Москва г, Зеленоград г, корпус 200Г, к...",...,NaN,NaN,Нет,2023-02-12,2023-03-04,61.0,Старше трудосп,NaN,макаркинагалинавладимировна1961-05-06,jul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87775,2023-03-29,Савченко Инна Ивановна,1927-05-21,Женский,2023-03-24,Умер,2023-03-31,Люблино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",...,NaN,NaN,Нет,2023-03-29,2023-03-31,95.0,Старше трудосп,NaN,савченкоиннаивановна1927-05-21,jul
87807,2023-03-30,Капенкин Виталий Егорович,1942-08-01,Мужской,2023-03-24,Умер,2023-03-30,Куркино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Южное ...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,80.0,Старше трудосп,NaN,капенкинвиталийегорович1942-08-01,jul
87810,2023-03-30,Комаров Юрий Сергеевич,1950-05-11,Мужской,NaT,Умер,2023-03-30,Дегунино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Левобе...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,72.0,Старше трудосп,NaN,комаровюрийсергеевич1950-05-11,jul
87815,2023-03-30,Медведева Лариса Ивановна,1937-04-16,Женский,2023-03-24,Умер,2023-03-31,Дегунино ОВПП,Москвич,"125565, Москва г, г, Ленинградское ш, дом 88,...",...,NaN,NaN,Нет,2023-03-30,2023-03-31,86.0,Старше трудосп,NaN,медведеваларисаивановна1937-04-16,jul


In [114]:
df_dead_1204

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ,period
0,2023-02-02,Зинатшина Анастасия Владимировна,1986-09-22,Женский,2023-01-31,Умер,2023-02-03,Зеленоград ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",...,NaN,NaN,Нет,2023-02-02,2023-02-03,36.0,NaN,NaN,зинатшинаанастасиявладимировна1986-09-22,apr
3,2023-03-03,Комарова Анастасия Анатольевна,1978-01-17,Женский,2023-03-01,Умер,2023-03-05,Зеленоград ОВПП,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",...,NaN,NaN,Нет,2023-03-03,2023-03-05,45.0,NaN,NaN,комароваанастасияанатольевна1978-01-17,apr
9,2023-03-19,Ямщикова Нина Ивановна,1969-03-22,Женский,2023-03-07,Умер,2023-03-27,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 847, квартира 83",...,NaN,NaN,Нет,2023-03-19,2023-03-27,54.0,NaN,NaN,ямщикованинаивановна1969-03-22,apr
12,2023-03-27,Кочетов Сергей Владимирович,1968-11-30,Мужской,2023-03-22,Умер,2023-03-28,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 1505, квартира 72",...,NaN,NaN,Нет,2023-03-27,2023-03-28,54.0,NaN,NaN,кочетовсергейвладимирович1968-11-30,apr
22,2023-02-12,Макаркина Галина Владимировна,1961-05-06,Женский,2023-02-03,Умер,2023-03-04,Зеленоград ОВПП,Москвич,"124681, Москва г, Зеленоград г, корпус 200Г, к...",...,NaN,NaN,Нет,2023-02-12,2023-03-04,61.0,Старше трудосп,NaN,макаркинагалинавладимировна1961-05-06,apr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87501,2023-03-29,Савченко Инна Ивановна,1927-05-21,Женский,2023-03-24,Умер,2023-03-31,Люблино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",...,NaN,NaN,Нет,2023-03-29,2023-03-31,95.0,Старше трудосп,NaN,савченкоиннаивановна1927-05-21,apr
87532,2023-03-30,Капенкин Виталий Егорович,1942-08-01,Мужской,2023-03-24,Умер,2023-03-30,Куркино,Москвич,"Москва г, вн.тер.г. муниципальный округ Южное ...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,80.0,Старше трудосп,NaN,капенкинвиталийегорович1942-08-01,apr
87535,2023-03-30,Комаров Юрий Сергеевич,1950-05-11,Мужской,NaT,Умер,2023-03-30,Дегунино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Левобе...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,72.0,Старше трудосп,NaN,комаровюрийсергеевич1950-05-11,apr
87541,2023-03-30,Медведева Лариса Ивановна,1937-04-16,Женский,2023-03-24,Умер,2023-03-31,Дегунино ОВПП,Москвич,"125565, Москва г, г, Ленинградское ш, дом 88,...",...,NaN,NaN,Нет,2023-03-30,2023-03-31,86.0,Старше трудосп,NaN,медведеваларисаивановна1937-04-16,apr


In [117]:
df_comp = pd.merge(df_dead_0407, df_dead_1204, how='left', on='Ключ')
df_comp

,Дата_вкл_реестр_x,Пациент_x,ДР_x,Пол_x,Дата_признания_x,Текущий статус пациента_x,Дата_прекр_x,Филиал_x,Иногород_x,Адрес_x,...,Адрес фактический_y,Дата разрешения ДЗМ_y,Номер разрешения ДЗМ_y,БК_y,Дата_первых_сведений_y,Дата_выбытия_y,Возраст_y,Категория_y,Дети_y,period_y
0,2023-02-02,Зинатшина Анастасия Владимировна,1986-09-22,Женский,2023-01-31,Умер,2023-02-03,Зеленоград ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",...,"124482, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2023-02-02,2023-02-03,36.0,NaN,NaN,apr
1,2023-03-03,Комарова Анастасия Анатольевна,1978-01-17,Женский,2023-03-01,Умер,2023-03-05,Зеленоград ОВПП,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",...,"124536, Москва г, Зеленоград г, корпус 511, кв...",NaN,NaN,Нет,2023-03-03,2023-03-05,45.0,NaN,NaN,apr
2,2023-03-19,Ямщикова Нина Ивановна,1969-03-22,Женский,2023-03-07,Умер,2023-03-27,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 847, квартира 83",...,"Москва г, Зеленоград г, корпус 847, квартира 8...",NaN,NaN,Нет,2023-03-19,2023-03-27,54.0,NaN,NaN,apr
3,2023-03-27,Кочетов Сергей Владимирович,1968-11-30,Мужской,2023-03-22,Умер,2023-03-28,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 1505, квартира 72",...,"Москва г, Зеленоград г, корпус 1505, квартира ...",NaN,NaN,Нет,2023-03-27,2023-03-28,54.0,NaN,NaN,apr
4,2023-02-12,Макаркина Галина Владимировна,1961-05-06,Женский,2023-02-03,Умер,2023-03-04,Зеленоград ОВПП,Москвич,"124681, Москва г, Зеленоград г, корпус 200Г, к...",...,"124681, Москва г, Зеленоград г, корпус 200Г, к...",NaN,NaN,Нет,2023-02-12,2023-03-04,61.0,Старше трудосп,NaN,apr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2023-03-29,Савченко Инна Ивановна,1927-05-21,Женский,2023-03-24,Умер,2023-03-31,Люблино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",...,"Москва г, вн.тер.г. муниципальный округ Марьин...",NaN,NaN,Нет,2023-03-29,2023-03-31,95.0,Старше трудосп,NaN,apr
3649,2023-03-30,Капенкин Виталий Егорович,1942-08-01,Мужской,2023-03-24,Умер,2023-03-30,Куркино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Южное ...",...,"Москва г, вн.тер.г. муниципальный округ Южное ...",NaN,NaN,Нет,2023-03-30,2023-03-30,80.0,Старше трудосп,NaN,apr
3650,2023-03-30,Комаров Юрий Сергеевич,1950-05-11,Мужской,NaT,Умер,2023-03-30,Дегунино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Левобе...",...,"125445, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2023-03-30,2023-03-30,72.0,Старше трудосп,NaN,apr
3651,2023-03-30,Медведева Лариса Ивановна,1937-04-16,Женский,2023-03-24,Умер,2023-03-31,Дегунино ОВПП,Москвич,"125565, Москва г, г, Ленинградское ш, дом 88,...",...,"Москва г, г, Ленинградское ш, дом 88, квартира...",NaN,NaN,Нет,2023-03-30,2023-03-31,86.0,Старше трудосп,NaN,apr


In [118]:
df_only_new = df_comp.loc[df_comp['period_y'].isna()]
df_only_new

,Дата_вкл_реестр_x,Пациент_x,ДР_x,Пол_x,Дата_признания_x,Текущий статус пациента_x,Дата_прекр_x,Филиал_x,Иногород_x,Адрес_x,...,Адрес фактический_y,Дата разрешения ДЗМ_y,Номер разрешения ДЗМ_y,БК_y,Дата_первых_сведений_y,Дата_выбытия_y,Возраст_y,Категория_y,Дети_y,period_y
173,NaT,Авдонкин Алексей Алексеевич (ДУБЛЬ),1951-03-08,Мужской,2022-12-21,Умер,2023-01-11,Координационный центр,NaN,"Москва г, вн.тер.г. муниципальный округ Митино...",...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
183,NaT,Бурцева Галина Анатольевна,1958-05-14,Женский,2023-01-17,Умер,2023-01-31,Куркино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Северн...",...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
189,NaT,Грибова Ирина Евгеньевна,1941-12-14,Женский,2022-12-07,Умер,2023-03-06,Дегунино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Коптев...",...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
243,NaT,Сулягин Валентин Васильевич,1939-09-25,Мужской,2023-03-10,Умер,2023-03-15,NaN,NaN,г Москва Волжский бульвар д 11 кв 22,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
321,2019-10-31,Захаров Александр Александрович,1953-11-21,Мужской,2019-10-30,Умер,2023-03-26,Царицыно ОВПП,Москвич,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
394,2020-12-25,Тельнова Наталья Павловна,1946-02-11,Женский,NaT,Умер,2023-03-24,Куркино ОВПП,Москвич,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
419,2021-03-11,Маслова Галина Георгиевна,1939-10-21,Женский,NaT,Умер,2023-03-11,Куркино ОВПП,Москвич,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
468,2021-07-08,Захарова Нина Михайловна,1931-05-29,Женский,NaT,Умер,2023-03-30,Бутово ОВПП,Москвич,Москва Северо-Восточный Лианозово Новгородск...,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
525,2021-10-11,Кортхонджиа Мурман Валикоевич,1957-08-03,Мужской,NaT,Умер,2023-03-27,Перово ОВПП,Москвич,"г.Москва, Старокирочный пер, д.16/2, корп.5, к...",...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
531,2021-10-20,Тараканова Тамара Григорьевна,1955-11-01,Женский,2021-09-09,Умер,2023-03-10,Черемушки ОВПП,Москвич,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [119]:
df_dead_1204.loc[df_dead_1204['Пациент'] =='Захаров Александр Александрович']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ,period


In [120]:
df_dead_0407.loc[df_dead_0407['Пациент'] =='Захаров Александр Александрович']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ,period
38292,2019-10-31,Захаров Александр Александрович,1953-11-21,Мужской,2019-10-30,Умер,2023-03-26,Царицыно ОВПП,Москвич,NaN,...,NaN,NaN,Нет,2019-10-31,2023-03-26,69.0,Старше трудосп,NaN,захаровалександралександрович1953-11-21,jul


In [102]:
df_only_new.loc[df_only_new['period_x'] =='jul']

,Дата_вкл_реестр_x,Пациент_x,ДР_x,Пол_x,Дата_признания_x,Текущий статус пациента_x,Дата_прекр_x,Филиал_x,Иногород_x,Адрес_x,...,Адрес фактический_y,Дата разрешения ДЗМ_y,Номер разрешения ДЗМ_y,БК_y,Дата_первых_сведений_y,Дата_выбытия_y,Возраст_y,Категория_y,Дети_y,period_y


In [105]:
df_dead_1204

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ,period
0,2023-02-02,Зинатшина Анастасия Владимировна,1986-09-22,Женский,2023-01-31,Умер,2023-02-03,Зеленоград ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",...,NaN,NaN,Нет,2023-02-02,2023-02-03,36.0,NaN,NaN,зинатшинаанастасиявладимировна1986-09-22,jul
3,2023-03-03,Комарова Анастасия Анатольевна,1978-01-17,Женский,2023-03-01,Умер,2023-03-05,Зеленоград ОВПП,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",...,NaN,NaN,Нет,2023-03-03,2023-03-05,45.0,NaN,NaN,комароваанастасияанатольевна1978-01-17,jul
9,2023-03-19,Ямщикова Нина Ивановна,1969-03-22,Женский,2023-03-07,Умер,2023-03-27,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 847, квартира 83",...,NaN,NaN,Нет,2023-03-19,2023-03-27,54.0,NaN,NaN,ямщикованинаивановна1969-03-22,jul
12,2023-03-27,Кочетов Сергей Владимирович,1968-11-30,Мужской,2023-03-22,Умер,2023-03-28,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 1505, квартира 72",...,NaN,NaN,Нет,2023-03-27,2023-03-28,54.0,NaN,NaN,кочетовсергейвладимирович1968-11-30,jul
22,2023-02-12,Макаркина Галина Владимировна,1961-05-06,Женский,2023-02-03,Умер,2023-03-04,Зеленоград ОВПП,Москвич,"124681, Москва г, Зеленоград г, корпус 200Г, к...",...,NaN,NaN,Нет,2023-02-12,2023-03-04,61.0,Старше трудосп,NaN,макаркинагалинавладимировна1961-05-06,jul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87501,2023-03-29,Савченко Инна Ивановна,1927-05-21,Женский,2023-03-24,Умер,2023-03-31,Люблино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",...,NaN,NaN,Нет,2023-03-29,2023-03-31,95.0,Старше трудосп,NaN,савченкоиннаивановна1927-05-21,jul
87532,2023-03-30,Капенкин Виталий Егорович,1942-08-01,Мужской,2023-03-24,Умер,2023-03-30,Куркино,Москвич,"Москва г, вн.тер.г. муниципальный округ Южное ...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,80.0,Старше трудосп,NaN,капенкинвиталийегорович1942-08-01,jul
87535,2023-03-30,Комаров Юрий Сергеевич,1950-05-11,Мужской,NaT,Умер,2023-03-30,Дегунино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Левобе...",...,NaN,NaN,Нет,2023-03-30,2023-03-30,72.0,Старше трудосп,NaN,комаровюрийсергеевич1950-05-11,jul
87541,2023-03-30,Медведева Лариса Ивановна,1937-04-16,Женский,2023-03-24,Умер,2023-03-31,Дегунино ОВПП,Москвич,"125565, Москва г, г, Ленинградское ш, дом 88,...",...,NaN,NaN,Нет,2023-03-30,2023-03-31,86.0,Старше трудосп,NaN,медведеваларисаивановна1937-04-16,jul


### 
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Функции для расчета ПОСЕЩЕНИЙ <br>
</b>            </div>

#### функция ВПС

In [149]:
def load_data_vps(file_name, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df_vps = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    
    df_vps = df_vps[['Взаимодействие.Вид взаимодействия',
           'Взаимодействие.Клиент.Наименование', 
                     # 'Взаимодействие.Ответственный',
           'Взаимодействие.Подразделение', 
                     'Клиент.БК', 
                     'Клиент.Наблюдение в РЦ',
           # 'Клиент.Адрес фактический',
                     'Клиент.Дата рождения',
           # 'Дата завершения взаимодействия', 
                     'Клиент.Пол',
           'Взаимодействие.Плановая дата', 
                     'Выдано_респ_оборуд'
                     # 'Количество записей'
                     
                    ]]
        #  переименуем покороче столбцы
    df_vps.rename(columns={'Взаимодействие.Вид взаимодействия': 'Вид_визита'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Клиент.Наименование': 'Пациент'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Подразделение': 'Филиал'}, inplace=True)
    df_vps.rename(columns={'Клиент.Наблюдение в РЦ': 'Набл_РЦ'}, inplace=True)
    df_vps.rename(columns={'Клиент.Дата рождения': 'ДР'}, inplace=True)
    df_vps.rename(columns={'Клиент.Пол': 'Пол'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Плановая дата': 'Дата_визита'}, inplace=True)
    df_vps.shape[0]   
        
                # --------------------старый  набор ВИЗИТОВ--------------------
    df_vps = df_vps.loc[(~df_vps['Вид_визита'].isin([
     #      'Первичный визит врача КЦ',
     #     'Первичный визит врача ОВПП', 
     # 'Первичный визит заведующего ОВПП',
    #     'Визит врача КЦ', 
        'Звонок - контроль состояния', 
        'Звонок врача - контроль состояния',
        'Звонок медицинской сестры - контроль состояния',
             'Звонок врача-подтверждение визита', 'Телефонный звонок', 'Итого'  
               'Звонок - консультация социального работника',            
           'Звонок-подтверждение визита', 
        'Визит младшей медицинской сестры',
           # ' Визит медбрата',
        'Визит социального работника', 'Итого',
        'Звонок - консультация социального работника',
             'Звонок - контроль получения НС',
        'Звонок врача - контроль состояния НЕ ИСПОЛЬЗОВАТЬ',
    ]))] 

    

    df_vps = df_vps.drop_duplicates(keep='first')


    
        # ----------------форматнем дату к дате
    df_vps['ДР'] = pd.to_datetime(df_vps['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_vps['Дата_визита'] = pd.to_datetime(df_vps['Дата_визита'], dayfirst=True, errors = 'coerce').dt.normalize()
    
       # ------------подсчитаем ВОЗРАСТ --------------на дату ФИНИША!!!

    # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_vps.loc[ (~df_vps['Дата_визита'].isnull()), 'Возраст'] = (df_vps['Дата_визита'] - df_vps['ДР']).dt.days//365

    
        #  ------ определим детей ---------------
    df_vps['Дети'] = np.nan
    df_vps.loc[(df_vps['Возраст'] < 18) & (df_vps['Возраст'] >= 0), 'Дети'] = 'Дети'
    
    
    
        #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_vps['Ключ'] = df_vps['Пациент'].map(str) + (df_vps['ДР'].dt.date).map(str)
    df_vps['Ключ'] = df_vps['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_vps['Ключ'] = df_vps['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
    

    return df_vps



### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования Датафрейма данных по выездам за период  <br></b>            </div>

In [150]:
def f_create_df_vizit_vps(file_name, start, finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df = load_data_vps(file_name, finish)
    
    df = df.loc[ (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )]
    
    #  ------------------------------------------Взрослые ВСЕГО -------
    df_vzrosl_all = df.loc[ 
                           (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )
                        &  ((df['Дети'].isna()))
                                &  (df['Клиент.БК'] != 'Да') 
                          ].shape[0]
    

    #  ------------------------------------------Взрослые Выезды ВРАЧЕЙ -------заведующе
    df_vzrosl_doctor = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        # &  (df['Вид_визита'].str.contains("врач") == True)   
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )      
                          ].shape[0] 

    #  ------------------------------------------Взрослые Выезды среднего м/п ------- медицинской 
    df_vzrosl_nurse = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))        
                          ].shape[0]   
    
     #  ------------------------------------------------------------------------Дети ВСЕГО -------
    df_deti_all = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~(df['Дети'].isna()))
                          ].shape[0]
    
    
    #  ------------------------------------------Дети  Выезды ВРАЧЕЙ -------
    df_deti_doctor = df.loc[ 
                          ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )     
                          ].shape[0]  
    
    #  ------------------------------------------Дети  Выезды среднего м/п -------
    df_deti_nurse = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))            
                          ].shape[0]  
    
    

    # return df
    return df_vzrosl_all, df_vzrosl_doctor, df_vzrosl_nurse, df_deti_all, df_deti_doctor, df_deti_nurse

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования Датафрейма данных по выездам к РЕСПИРАТОРНЫМ за период  <br></b>            </div>

In [151]:
# ------------------------------------------------------------------------------------ по признаку "1"--------------------------
def f_create_df_vizit_vps_k_resper(file_name, start, finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df = load_data_vps(file_name,finish)
    df = df.loc[ (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )]
    
    #  ------------------------------------------Взрослые ВСЕГО -------
    df_vzrosl_all_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        # &  (df['Клиент.БК'] != 'Да') 
        
                          ].shape[0]


    #  ------------------------------------------Взрослые Выезды ВРАЧЕЙ -------заведующе
    df_vzrosl_doctor_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )      
                          ].shape[0] 

    #  ------------------------------------------Взрослые Выезды среднего м/п ------- медицинской 
    df_vzrosl_nurse_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))        
                          ].shape[0]   
    
     #  ------------------------------------------------------------------------Дети ВСЕГО -------
    df_deti_all_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                         &  (~(df['Дети'].isna()))
                         &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                          ].shape[0]
    
    
    #  ------------------------------------------Дети  Выезды ВРАЧЕЙ -------
    df_deti_doctor_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )     
                          ].shape[0]  
    
    #  ------------------------------------------Дети  Выезды среднего м/п -------
    df_deti_nurse_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))            
                          ].shape[0]  
    


    # return df
    df_vzrosl_all_resper_df = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())
                        &  ((df['Выдано_респ_оборуд'] != '0') | (df['Набл_РЦ'] != 'Нет'))
                          ]
    return df_vzrosl_all_resper, df_vzrosl_doctor_resper, df_vzrosl_nurse_resper, df_deti_all_resper, df_deti_doctor_resper, df_deti_nurse_resper, df_vzrosl_all_resper_df

#
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font color= green size=10>
<b> ВЫВОД РЕЗУЛЬТАТОВ по выездам <br>
</b>            </div>

In [152]:
start = '2023-01-01'
finish = '2023-07-01'
# file_name = 'впс_1й_кв_0704.csv'
# file_name = 'впс_1й_кв_0704_выданое_респ.csv'
file_name = 'впс_1е_полугод_с_выданным_респ.csv'

df_vps = f_create_df_vizit_vps(file_name, start, finish)
df_resp = f_create_df_vizit_vps_k_resper(file_name, start, finish)

a = f'Совершено выездов ВПС в период {start} - {finish}:'
display_markdown(f'''# {a}
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __{df_vps[0]}__ |__{df_vps[0]}__ |__{df_resp[0]}__ |
| Совершено выездов врачами  (к взрослым) | __{df_vps[1]}__ |__{df_vps[1]}__ |__{df_resp[1]}__ |
| Совершено выездов сред.м/п (к взрослым) | __{df_vps[2]}__ |__{df_vps[3]}__ |__{df_resp[2]}__ |
| Совершено выездов к детям               | __{df_vps[3]}__ |__{df_vps[3]}__ |__{df_resp[3]}__ |
| Совершено выездов врачами  (к детям)    | __{df_vps[4]}__ |__{df_vps[4]}__ |__{df_resp[4]}__ |
| Совершено выездов сред.м/п (к детям)    | __{df_vps[5]}__ |__{df_vps[5]}__ |__{df_resp[5]}__ |

''', raw=True)

# Совершено выездов ВПС в период 2023-01-01 - 2023-07-01:
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __117964__ |__117964__ |__8110__ |
| Совершено выездов врачами  (к взрослым) | __45743__ |__45743__ |__4659__ |
| Совершено выездов сред.м/п (к взрослым) | __72317__ |__2177__ |__3451__ |
| Совершено выездов к детям               | __2177__ |__2177__ |__1005__ |
| Совершено выездов врачами  (к детям)    | __1336__ |__1336__ |__556__ |
| Совершено выездов сред.м/п (к детям)    | __841__ |__841__ |__449__ |



In [153]:
start = '2023-01-01'
finish = '2023-07-01'
# file_name = 'впс_1й_кв_0704.csv'
# file_name = 'впс_1й_кв_0704_выданое_респ.csv'
file_name = 'впс_1е_полугод_с_выданным_респ_без_аспират.csv'

df_vps = f_create_df_vizit_vps(file_name, start, finish)
df_resp = f_create_df_vizit_vps_k_resper(file_name, start, finish)

a = f'Совершено выездов ВПС в период {start} - {finish}:'
display_markdown(f'''# {a}
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой(АСПИРАТОРЫ не РЕСПИРАТОРКА) |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __{df_vps[0]}__ |__{df_vps[0]}__ |__{df_resp[0]}__ |
| Совершено выездов врачами  (к взрослым) | __{df_vps[1]}__ |__{df_vps[1]}__ |__{df_resp[1]}__ |
| Совершено выездов сред.м/п (к взрослым) | __{df_vps[2]}__ |__{df_vps[3]}__ |__{df_resp[2]}__ |
| Совершено выездов к детям               | __{df_vps[3]}__ |__{df_vps[3]}__ |__{df_resp[3]}__ |
| Совершено выездов врачами  (к детям)    | __{df_vps[4]}__ |__{df_vps[4]}__ |__{df_resp[4]}__ |
| Совершено выездов сред.м/п (к детям)    | __{df_vps[5]}__ |__{df_vps[5]}__ |__{df_resp[5]}__ |

''', raw=True)

# Совершено выездов ВПС в период 2023-01-01 - 2023-07-01:
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой(АСПИРАТОРЫ не РЕСПИРАТОРКА) |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __117964__ |__117964__ |__7526__ |
| Совершено выездов врачами  (к взрослым) | __45743__ |__45743__ |__4387__ |
| Совершено выездов сред.м/п (к взрослым) | __72317__ |__2177__ |__3139__ |
| Совершено выездов к детям               | __2177__ |__2177__ |__742__ |
| Совершено выездов врачами  (к детям)    | __1336__ |__1336__ |__419__ |
| Совершено выездов сред.м/п (к детям)    | __841__ |__841__ |__323__ |



### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Расчет сведений по ДИАГНОЗАМ из 605го <br>
</b>            </div>

###
<div class="alert alert-block alert-info">

<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font size=3>
<b>Распишем ДИАГНОЗЫ (внесем новые колонки-принаки в датафрейм)</b>               

In [154]:
def diagnoz(df_podgotovlen):
    
         #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------

    df_concat = df_podgotovlen
    # ----- к нижнему регистру, все, что  в Основной диагноз
    df_concat['Диагноз'] = df_concat['Диагноз'].str.lower()

    # ---- новый Признак ДИАГНОЗА проставлен-----
    df_concat["признак_Диагноза"] = np.nan

    # ---------------------------------------------------------------- новый признак Онкология-----
    df_concat["Onko"] = np.nan

    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("рак") == True)           
              |(df_concat['Диагноз'].str.contains("зно") == True)           
              |(df_concat['Диагноз'].str.contains("образовани") == True)           
              |(df_concat['Диагноз'].str.contains("онкол") == True)           
              |(df_concat['Диагноз'].str.contains("миелом") == True)           
              |(df_concat['Диагноз'].str.contains("опухо") == True)    
              |(df_concat['Диагноз'].str.contains("мтс") == True)  
              |(df_concat['Диагноз'].str.contains("пммр") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("mts") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("сарко") == True)  


              |(df_concat['Диагноз'].str.contains("лейкоз") == True)  
              |(df_concat['Диагноз'].str.contains("c00") == True)  
              |(df_concat['Диагноз'].str.contains("c97") == True)  
              |(df_concat['Диагноз'].str.contains("d48") == True) 


                  |(df_concat['Диагноз'].str.contains("с00") == True)  
              |(df_concat['Диагноз'].str.contains("с97") == True)  
              |(df_concat['Диагноз'].str.contains("d48") == True) 

              |(df_concat['Диагноз'].str.contains("c43") == True) 
              |(df_concat['Диагноз'].str.contains("d32") == True) 
              |(df_concat['Диагноз'].str.contains("c16") == True) 
              |(df_concat['Диагноз'].str.contains("c34") == True) 

         |(df_concat['Диагноз'].str.contains("с80") == True) 
         |(df_concat['Диагноз'].str.contains("с18") == True) 
         |(df_concat['Диагноз'].str.contains("глиобластома головного мозга") == True)  

                    # |(df_gosp['Диагноз'].str.contains("d48") == True)  
                    # |(df_gosp['Диагноз'].str.contains("d48") == True)  
    , 'Onko' ] = 'Onko'

    df_concat.loc[~df_concat['Onko'].isna(), 'признак_Диагноза'] = 'Есть'





    # --------------------------------------------------------------------- новый признак Неврология-----
    df_concat["Неврология"] = np.nan

    # ---- --------------------------- признак Неврология-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("неврология") == True)           
              |(df_concat['Диагноз'].str.contains("g00 - g99") == True)           
              |(df_concat['Диагноз'].str.contains("g00-g98") == True)           
              |(df_concat['Диагноз'].str.contains("g00") == True)           
              |(df_concat['Диагноз'].str.contains("болезни нервной системы") == True)           


    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    , 'Неврология' ] = 'Неврология'

    df_concat.loc[~df_concat['Неврология'].isna(), 'признак_Диагноза'] = 'Есть'

    # ---------------------------------------------------------------------- новый признак Терапия-----
    df_concat["Терапия"] = np.nan

    # ---------------------------------------- новый признак Терапия-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains(" болезни системы кровообращения") == True)  
               | (df_concat['Диагноз'].str.contains("i00-i99") == True)

               | (df_concat['Диагноз'].str.contains("k00-k93") == True)
               | (df_concat['Диагноз'].str.contains("болезни органов пищеварения") == True)

               | (df_concat['Диагноз'].str.contains("j00-j99") == True)
               | (df_concat['Диагноз'].str.contains("болезни органов дыхания") == True)

               | (df_concat['Диагноз'].str.contains("m00-m99") == True)
               | (df_concat['Диагноз'].str.contains("костно-мышечной") == True)

               | (df_concat['Диагноз'].str.contains("n00-n99") == True)
               | (df_concat['Диагноз'].str.contains("мочеполовой системы") == True)

               | (df_concat['Диагноз'].str.contains("d82") == True)

               | (df_concat['Диагноз'].str.contains("d50-d89") == True)
               | (df_concat['Диагноз'].str.contains("болезни крови, кроветворных") == True)

               | (df_concat['Диагноз'].str.contains("цвб хигм") == True)

               | (df_concat['Диагноз'].str.contains("м00-м050") == True)

    #            | (df_concat['Диагноз'].str.contains("м00-м050") == True)
    #            | (df_concat['Диагноз'].str.contains("болезни органов дыхания") == True)




    , 'Терапия' ] = 'Терапия'

    df_concat.loc[~df_concat['Терапия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ---------------------------------------------------------------------------- новый признак Педиатрия-----
    df_concat["Педиатрия"] = np.nan

    # --------------------------------------- новый признак Педиатрия-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("p00-p96") == True)  
               | (df_concat['Диагноз'].str.contains("отдельные состояния, возникающие") == True)
               | (df_concat['Диагноз'].str.contains("р00-р96") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)
               # | (df_concat['Диагноз'].str.contains("травмы, отравления и некоторые") == True)

        , 'Педиатрия' ] = 'Педиатрия'

    df_concat.loc[~df_concat['Педиатрия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Врожденные_аномалии-----  
    df_concat["Врожденные"] = np.nan

    # --------------------------------------- новый признак Врожденные_аномалии-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("q00-q99") == True)  
               | (df_concat['Диагноз'].str.contains("пороки развития") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)

        , 'Врожденные' ] = 'Врожденные'

    df_concat.loc[~df_concat['Врожденные'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак ВИЧ----- 
    df_concat["ВИЧ"] = np.nan

    # --------------------------------------- новый признак ВИЧ-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("в20.7") == True)  
               | (df_concat['Диагноз'].str.contains("вич-инфекция") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)

        , 'ВИЧ' ] = 'ВИЧ'

    df_concat.loc[~df_concat['ВИЧ'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Психиатрия-----
    df_concat["Психиатрия"] = np.nan


    # --------------------------------------- новый признак Психиатрия-----  
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("f00-f99") == True)  
               | (df_concat['Диагноз'].str.contains("психические расстройства") == True)

               | (df_concat['Диагноз'].str.contains("f.20") == True)
               | (df_concat['Диагноз'].str.contains("f.02") == True)


               | (df_concat['Диагноз'].str.contains("шизофрения") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("f.20.004) шизофрения") == True)

        , 'Психиатрия' ] = 'Психиатрия'

    df_concat.loc[~df_concat['Психиатрия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Прочие-----   
    df_concat["Прочие"] = np.nan

    # --------------------------------------- новый признак Прочие-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("инфекционные и паразитарные болезни") == True)  
               | (df_concat['Диагноз'].str.contains("a00-b99") == True)
               | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               | (df_concat['Диагноз'].str.contains("s00-t98") == True)
               | (df_concat['Диагноз'].str.contains("травмы, отравления и некоторые") == True)
               | (df_concat['Диагноз'].str.contains("короновирусная инфекция") == True)
               | (df_concat['Диагноз'].str.contains("r00-r99") == True)
               | (df_concat['Диагноз'].str.contains("l00-l99") == True)
               | (df_concat['Диагноз'].str.contains("болезни кожи и подкожной клетчатки") == True)
               | (df_concat['Диагноз'].str.contains("h00-h59") == True)
               | (df_concat['Диагноз'].str.contains("e00-e90") == True)
               | (df_concat['Диагноз'].str.contains("эндокринной системы") == True)
               | (df_concat['Диагноз'].str.contains("алкогольный цирроз") == True)


              | (df_concat['Диагноз'].str.contains("е00-е90") == True)


    #            | (df_concat['Диагноз'].str.contains("е00-е90") == True)
    #            | (df_concat['Диагноз'].str.contains("костно-мышечной") == True)

        , 'Прочие' ] = 'Прочие'

    df_concat.loc[~df_concat['Прочие'].isna(), 'признак_Диагноза'] = 'Есть'

    # ------------------------ после всех ПРИЗНАКОВ остатки - в ПРОЧИЕ ----
    df_concat.loc[df_concat['признак_Диагноза'].isna(), 'Прочие'] = 'Прочие'

    df_concat.loc[~df_concat['Прочие'].isna(), 'признак_Диагноза'] = 'Есть'

    return df_concat

###
<div class="alert alert-block alert-warning">

<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> <font size=3>
<b> функция подсчета пациентов по ВИДУ диагноза  </b>               

#### ----------------------- функция ДИАГНОЗА --------------

In [155]:
# ----------------------- функция расчета Неврология--------------
def calculate_count_diagnoz(df, start, finish, diagnoz):
    #     ---------------------------------------------------------------- ВЗРОСЛЫХ ----------------------------------------------------------
    diagnoz = diagnoz
    vzros = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                          & (  df[diagnoz] == diagnoz)                                                                  # ОНКО = ДА
                          & ( (df['Дети']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]
    
    
    vzros_trud = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                          & (  df[diagnoz] == diagnoz)                                                                  # ОНКО = ДА
                          & ( (df['Дети']).isna())                                                                         # Дети = НЕТ 
                          & ( ~(df['Категория']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]
    
    
    
    
    
    
    
    #     ---------------------------------------------------------------- Дети ----------------------------------------------------------
    deti = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                         
                          & (  df[diagnoz] == diagnoz)                                                                 # ОНКО = No
                          & ( ~(df['Дети']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]    
    
    return vzros, vzros_trud, deti    

In [156]:
# --------------------------------------------------------------------------------функция подсчета ВСЕГО пациентов --------------
#--------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients(df, start, finish):

    # data_all_clients = df.loc[ (df['Дата_вкл_реестр'] <= finish)  
    #                          &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )].shape[0] 

    
    data_all_clients_vzros = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                  & ( (df['Дети'].isnull()))   
        
                       ].shape[0] 
    
    data_all_clients_vzros_trud = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                  &  df['Дети'].isnull()
                                  & ( ~(df['Категория'].isnull()))
                                      ].shape[0] 
        
        
    
    data_all_clients_deti = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                 & ( ~(df['Дети'].isnull()))   
        
                       ].shape[0] 
    
    return  data_all_clients_vzros, data_all_clients_vzros_trud, data_all_clients_deti

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Результат подсчета ДИАГНОЗОВ в периоде <br></b>            </div>

In [157]:
def poluchateli_v_reestre(name_605, name_preparat, start, finish, bmchu):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()


    return df_605_period.shape[0], df_605_period_trud, df_preparat_vzros.shape[0], df_preparat_vzros_trud.shape[0], df_preparat_deti.shape[0]
    

####
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция Получатели (ДЛЯ ДАТАФРЕЙМОВ) <br></b>            </div>

In [158]:
def poluchateli_v_reestre_df(name_605, name_preparat, start, finish, bmchu):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()


    return df_605_period, df_605_period_trud, df_preparat_vzros, df_preparat_vzros_trud, df_preparat_deti, df_preparat

<hr>

<font color= green size=10>__ВЫВОД РЕЗУЛЬТАТОВ__
<hr>


In [161]:
# ---------------------------- ИТОГИ ---------------
start = '2023-01-01'
finish = '2023-07-01'

# file_name = '605_рц_бк_1204.csv'
file_name = '605_рц_бк_0407.csv'

bmchu = 'не учитываем'
# bmchu = 'учитываем'
bk = 'с учетом БК'
# bk = 'без учета БК'


# ---------------------------- Готовим датафрейм РЕЕСТРА ---------------
df_reestr = load_data_605(file_name, bmchu, bk,finish)

# ---------------------------- Готовим датафрейм c РАСПИСАНЫМИ ДИАГНОЗАМИ ---------------
df_insert_diagnoz = diagnoz(df_reestr)

s_list = ['Onko', 'Неврология', 'Терапия',  'Педиатрия', 'Врожденные', 'ВИЧ', 'Психиатрия',  'Прочие']

#  ------------------- считаем в цикле пациентов для  ['Неврология', 'Терапия',  'Педиатрия', 'Врожденные', 'ВИЧ', 'Психиатрия',  'Прочие'] --
list_diagn_pac = []
for diagn in s_list:
    list_diagn_pac.append(calculate_count_diagnoz(df_insert_diagnoz, start, finish, diagn))
    

    
data_stat_all = def_data_all_clients(df_insert_diagnoz, start, finish)
    
god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]

a = f'Статистика по ДИАГНОЗАМ в период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f'''
# {a}
| Параметр | Взрослые | В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|
| __Всего пациентов__{bk}__        | {data_stat_all[0]} | {data_stat_all[1]} |{data_stat_all[2]} |
| пациенты с диагнозом __"Онко"__  | {list_diagn_pac[0][0]} | {list_diagn_pac[0][1]} |{list_diagn_pac[0][2]} |
| пациенты с диагнозом __"Неврология"__  | {list_diagn_pac[1][0]} | {list_diagn_pac[1][1]} |{list_diagn_pac[1][2]} |
| пациенты с диагнозом __"Терапия"__  | {list_diagn_pac[2][0]} | {list_diagn_pac[2][1]} |{list_diagn_pac[2][2]} |
| пациенты с диагнозом __"Педиатрия"__  | {list_diagn_pac[3][0]} | {list_diagn_pac[3][1]} |{list_diagn_pac[3][2]} |
| пациенты с диагнозом __"Врожденные"__  | {list_diagn_pac[4][0]} | {list_diagn_pac[4][1]} |{list_diagn_pac[4][2]} |
| пациенты с диагнозом __"ВИЧ"__  | {list_diagn_pac[5][0]} | {list_diagn_pac[5][1]} |{list_diagn_pac[5][2]} |
| пациенты с диагнозом __"Психиатрия"__  | {list_diagn_pac[6][0]} | {list_diagn_pac[6][1]} |{list_diagn_pac[6][2]} |
| пациенты с диагнозом __"Прочие"__  | {list_diagn_pac[7][0]} | {list_diagn_pac[7][1]} |{list_diagn_pac[7][2]} |
| __Статус по БМЧУ__  | __{bmchu}__ | --|-- |-- |




''', raw=True)


# Статистика по ДИАГНОЗАМ в период с 01.01.2023 по 01.07.2023:
| Параметр | Взрослые | В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|
| __Всего пациентов__с учетом БК__        | 19716 | 16789 |513 |
| пациенты с диагнозом __"Онко"__  | 10063 | 8501 |17 |
| пациенты с диагнозом __"Неврология"__  | 1807 | 1272 |354 |
| пациенты с диагнозом __"Терапия"__  | 6383 | 5872 |11 |
| пациенты с диагнозом __"Педиатрия"__  | 0 | 0 |15 |
| пациенты с диагнозом __"Врожденные"__  | 8 | 0 |95 |
| пациенты с диагнозом __"ВИЧ"__  | 6 | 0 |0 |
| пациенты с диагнозом __"Психиатрия"__  | 70 | 55 |7 |
| пациенты с диагнозом __"Прочие"__  | 1379 | 1089 |14 |
| __Статус по БМЧУ__  | __не учитываем__ | --|-- |-- |






In [ ]:
data_stat_all[4]

In [87]:
data_stat_all[4].to_excel('data_605_2023_бмчу.xlsx')